In [ ]:
# default_exp utils.engine

In [ ]:
#export
import os
import time
import timeit
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from abc import ABC
from fastprogress.fastprogress import master_bar, progress_bar

In [ ]:
#export
class Fitter(ABC):
    def __init__(self):
        pass
    
    def fit(self):
        pass
    
    def log(self):
        pass
    
    def train(self):
        pass
    
    def validate(self):
        pass

In [ ]:
#export
class BertFitterSimple(Fitter):
    #a simplified bertfitter. However, the model must calculate the loss so use model.EntityModelWithLoss
    def __init__(self, model, dataloaders, optimizer, metrics, device, log_file='training_log.txt',scheduler=None, trial=None):
        self.model = model
        self.train_dl, self.valid_dl = dataloaders[0], dataloaders[1]
        self.optimizer = optimizer
        self.scheduler = scheduler
        if not os.path.exists(os.path.join('..', 'outputs')): os.makedirs(os.path.join('..', 'outputs'))
        if os.path.exists(os.path.join('..', 'outputs', f'{log_file}')): 
            os.remove(os.path.join('..', 'outputs', f'{log_file}'))
        self.log_file = os.path.join('..', 'outputs', f'{log_file}')
        if not isinstance(metrics, (list, tuple)): 
            metrics = list(metrics)
        self.metrics = metrics
        self.device = device
        self.trial = trial #for optuna
        
    def fit(self, epochs, return_metric=False, 
            monitor='epoch train_loss valid_loss time', 
            model_path=os.path.join('..', 'weights', 'model.pth'), show_graph=True):
        self.model_path = model_path
        self.log(f'{time.ctime()}')
        self.log(f'Using device: {self.device}')
        mb = master_bar(range(1, epochs+1)) #MAJOR
        mb.write(monitor.split(),table=True)
        
        model = self.model.to(self.device)
        optimizer = self.optimizer
        scheduler = self.scheduler
        best_metric = -np.inf
        train_loss_list, valid_loss_list = [], []
        
        for i_, epoch in enumerate(mb):
            epoch_start = timeit.default_timer()
            start = time.time()
            self.log('-'*50)
            self.log(f'Running Epoch #{epoch} {"🔥"*epoch}')
            self.log(f'{"-"*50} \n')
            
            self.log('TRAINING...')
            train_loss = self.train(mb, model, optimizer, self.device, scheduler)
            train_loss_list.append(train_loss) #for graph
            self.log(f'Training time: {round(time.time()-start, 2)} secs \n')
            
            start = time.time()
            self.log('EVALUATING...')
            valid_loss = self.validate(mb, model, self.device)    
            valid_loss_list.append(valid_loss) #for graph
            
            if show_graph:
                self.plot_loss_update(epoch, epochs, mb, train_loss_list, valid_loss_list) # for graph
                               
            epoch_end = timeit.default_timer()
            total_time = epoch_end - epoch_start
            mins, secs = divmod(total_time, 60)
            hours, mins = divmod(mins, 60)
            ret_time = f'{int(hours)}:{int(mins)}:{int(secs)}'
            mb.write([epoch,f'{train_loss:.6f}',f'{valid_loss:.6f}',
                      f'{ret_time}'],table=True)
            self.log(f'Evaluation time: {ret_time}\n')
#             break
            
        if return_metric: return best_metric
    
    def train(self, mb, model, opt, device, sched=None):     
        model.train()
        train_loss = 0       
        for ind, xy in enumerate(progress_bar(self.train_dl, parent=mb)):
            y_tag = xy.pop('target_tag')
            y_pos = xy.pop('target_pos')
            x = xy
            inputs, target_tag, target_pos = {key: x_.to(device) for key, x_ in x.items()}, y_tag.to(device), y_pos.to(device)
            opt.zero_grad()
            *out, loss = model(**inputs, target_tag=y_tag, target_pos=y_pos)
            loss.backward()
            opt.step()       
            if sched is not None:
                sched.step()
            train_loss += loss.item()
            
            if ind % 500 == 0:
                self.log(f'Batch: {ind}, Train loss: {train_loss/ len(self.train_dl)}')
            
            mb.child.comment = f'{train_loss / (ind+1) :.3f}'
        return train_loss / len(self.train_dl)
    
    def validate(self, mb, model, device):    
        model.eval()
        valid_loss = 0      
        with torch.no_grad():
            for ind, xy in enumerate(progress_bar(self.valid_dl, parent=mb)):
                y_tag = xy.pop('target_tag')
                y_pos = xy.pop('target_pos')
                x = xy
                inputs, target_tag, target_pos = {key: x_.to(device) for key, x_ in x.items()}, y_tag.to(device), y_pos.to(device)
                *out, loss = model(**inputs, target_tag=y_tag, target_pos=y_pos)
                
                valid_loss += loss.item()

                if ind % 500 == 0:
                    self.log(f'Batch: {ind}, Valid loss: {valid_loss / (ind+1) :.3f}')

                mb.child.comment = f'{valid_loss / (ind+1) :.3f}'  
        return valid_loss / len(self.valid_dl)
        
            
    def log(self, message, verbose=False):
        if verbose: print(message)
        with open(self.log_file, 'a+') as logger_:
            logger_.write(f'{message}\n')
            
    
    @staticmethod
    def plot_loss_update(epoch, epochs, mb, train_loss, valid_loss):
        """ dynamically print the loss plot during the training/validation loop.
            expects epoch to start from 1.
        """
        x = range(1, epoch+1)
        y = np.concatenate((train_loss, valid_loss))
        graphs = [[x,train_loss], [x,valid_loss]]
        x_margin = 0.2
        y_margin = 0.05
        x_bounds = [1-x_margin, epochs+x_margin]
        y_bounds = [np.min(y)-y_margin, np.max(y)+y_margin]

        mb.update_graph(np.array(graphs), np.array(x_bounds), np.array(y_bounds))

In [ ]:
#export
class BertFitter(Fitter):
    def __init__(self, model, dataloaders, optimizer, metrics, device, log_file='training_log.txt',scheduler=None, trial=None):
        self.model = model
        self.train_dl, self.valid_dl = dataloaders[0], dataloaders[1]
        self.optimizer = optimizer
        self.scheduler = scheduler
        if not os.path.exists(os.path.join('..', 'outputs')): os.makedirs(os.path.join('..', 'outputs'))
        if os.path.exists(os.path.join('..', 'outputs', f'{log_file}')): 
            os.remove(os.path.join('..', 'outputs', f'{log_file}'))
        self.log_file = os.path.join('..', 'outputs', f'{log_file}')
        if not isinstance(metrics, (list, tuple)): 
            metrics = list(metrics)
        self.metrics = metrics
        self.device = device
        self.trial = trial #for optuna
        
    def fit(self, epochs, return_metric=False, 
            monitor='epoch train_loss valid_loss tag_accuracy tag_f1_score pos_accuracy pos_f1_score time', 
            model_path=os.path.join('..', 'weights', 'model.pth'), show_graph=True):
        self.model_path = model_path
        self.log(f'{time.ctime()}')
        self.log(f'Using device: {self.device}')
        mb = master_bar(range(1, epochs+1)) #MAJOR
        mb.write(monitor.split(),table=True)
        
        model = self.model.to(self.device)
        optimizer = self.optimizer
        best_metric = -np.inf
        train_loss_list, valid_loss_list = [], []
        
        for i_, epoch in enumerate(mb):
            epoch_start = timeit.default_timer()
            start = time.time()
            self.log('-'*50)
            self.log(f'Running Epoch #{epoch} {"🔥"*epoch}')
            self.log(f'{"-"*50} \n')
            self.log('TRAINING...')
            
            train_loss, valid_loss = 0, 0
            valid_metric_0_tag, valid_metric_1_tag =  0, 0
            valid_metric_0_pos, valid_metric_1_pos =  0, 0
            for ind, batch in enumerate(progress_bar(self.train_dl, parent=mb)):
                train_loss += self.train(batch, model, optimizer, self.device, self.scheduler)
                if ind % 500 == 0:
                    self.log(f'Batch: {ind}, Train loss: {train_loss / (ind+1) :.3f}')
#                 break
                mb.child.comment = f'{train_loss / (ind+1) :.3f}'
            train_loss /= mb.child.total
        
            train_loss_list.append(train_loss) #for graph
            self.log(f'Training time: {round(time.time()-start, 2)} secs \n')
            
            start = time.time()
            self.log('EVALUATING...')
            with torch.no_grad():
                for ind, batch in enumerate(progress_bar(self.valid_dl, parent=mb)):
                    valid_loss_, valid_metric_ = self.validate(batch, model, self.device)
                    valid_loss += valid_loss_
                    valid_metric_0_tag += valid_metric_[0]
                    valid_metric_1_tag += valid_metric_[1]
                    valid_metric_0_pos += valid_metric_[2]
                    valid_metric_1_pos += valid_metric_[3]
                    if ind % 500 == 0:
                        self.log(f'Batch: {ind}, Valid loss: {valid_loss / (ind+1) :.3f}')
#                     break   
                    mb.child.comment = f'{valid_loss / (ind+1) :.3f}'
    
            valid_loss /= mb.child.total
            valid_metric_0_tag /= mb.child.total
            valid_metric_1_tag /= mb.child.total
            valid_metric_0_pos /= mb.child.total
            valid_metric_1_pos /= mb.child.total
            valid_loss_list.append(valid_loss) #for graph
            
            if valid_metric_1_tag > best_metric: #ie (f1_score > inf)
                #             save model
                if self.model_path is not None:
                    if not os.path.exists(os.path.join('..', 'weights')): os.makedirs(os.path.join('..', 'weights'))
                    self.log(f'Saving model weights at {self.model_path}')
                    torch.save(model.state_dict(), self.model_path)
                best_metric = valid_metric_0_pos
                    
            if self.trial is not None:
                self.trial.report(best_metric, epoch)

                # Handle pruning based on the intermediate value.
                if self.trial.should_prune():
                    raise optuna.exceptions.TrialPruned()
            
            if show_graph:
                self.plot_loss_update(epoch, epochs, mb, train_loss_list, valid_loss_list) # for graph
                               
            epoch_end = timeit.default_timer()
            total_time = epoch_end - epoch_start
            mins, secs = divmod(total_time, 60)
            hours, mins = divmod(mins, 60)
            ret_time = f'{int(hours)}:{int(mins)}:{int(secs)}'
            mb.write([epoch,f'{train_loss:.6f}',f'{valid_loss:.6f}',
                      f'{valid_metric_0_tag:.6f}', f'{valid_metric_1_tag:.6f}', 
                      f'{valid_metric_0_pos:.6f}', f'{valid_metric_1_pos:.6f}', 
                      f'{ret_time}'],table=True)
            self.log(f'Evaluation time: {ret_time}\n')
#             break
            
        if return_metric: return best_metric
    
    def train(self, xy, model, opt, device, sched=None):
        model.train()
        y_tag = xy.pop('target_tag')
        y_pos = xy.pop('target_pos')
        x = xy
        inputs, target_tag, target_pos = [x_.to(device) for x_ in x.values()], y_tag.to(device), y_pos.to(device)
        opt.zero_grad()
        out = model(*inputs)
        loss_tag = self.loss_func(out[0], target_tag, x['attention_mask'], model.num_tag)
        loss_pos = self.loss_func(out[1], target_pos, x['attention_mask'], model.num_pos)
        loss = (loss_tag + loss_pos) / 2
        loss.backward()
        opt.step()       
        if sched is not None:
            sched.step()
        return loss.item()
    
    def validate(self, xy, model, device):
        model.eval()
        y_tag = xy.pop('target_tag')
        y_pos = xy.pop('target_pos')
        x = xy
        inputs, target_tag, target_pos = [x_.to(device) for x_ in x.values()], y_tag.to(device), y_pos.to(device)
        out = model(*inputs)
        loss_tag = self.loss_func(out[0], target_tag, x['attention_mask'], model.num_tag)
        loss_pos = self.loss_func(out[1], target_pos, x['attention_mask'], model.num_pos)
        loss = (loss_tag + loss_pos) / 2
        
#         skelarn metrics to be calculated for every item in batch
        cleaned_out_tag = out[0].cpu().softmax(2).argmax(dim=2) #[bs, seq_len, hidden_dim(num_labels)] -> [bs, seq_len]
        cleaned_out_pos = out[1].cpu().softmax(2).argmax(dim=2) #[bs, seq_len, hidden_dim(num_labels)] -> [bs, seq_len]
        all_metric_tag, all_metric_pos = [], []
        for i in range(target_tag.shape[0]):
            metric_0_tag = self.process_metric(target_tag, cleaned_out_tag, self.metrics[0], x, i)
            metric_1_tag = self.process_metric(target_tag, cleaned_out_tag, self.metrics[1], x, i)
            all_metric_tag.append([metric_0_tag, metric_1_tag])

            metric_0_pos = self.process_metric(target_pos, cleaned_out_pos, self.metrics[0], x, i)#sklearn metrics are (targ, inp)
            metric_1_pos = self.process_metric(target_pos, cleaned_out_pos, self.metrics[1], x, i)
            all_metric_pos.append([metric_0_pos, metric_1_pos])
        
        all_metric_tag = np.array(all_metric_tag)
        all_metric_pos = np.array(all_metric_pos)
        metrics = ((sum(all_metric_tag[:, 0]) / target_tag.shape[0]),
                   (sum(all_metric_tag[:, 1]) / target_tag.shape[0]),
                   (sum(all_metric_pos[:, 0]) / target_tag.shape[0]),
                   (sum(all_metric_pos[:, 1]) / target_tag.shape[0]))
        
        return loss.item(), metrics
            
    def log(self, message, verbose=False):
        if verbose: print(message)
        with open(self.log_file, 'a+') as logger_:
            logger_.write(f'{message}\n')
            
    @staticmethod
    def process_metric(target, output, metric, x, i):
        return metric(target[i].cpu()[torch.nonzero(x['attention_mask'][i]).flatten()], 
                        output[i][torch.nonzero(x['attention_mask'][i]).flatten()])
           
    @staticmethod
    def loss_func(out, target, mask, num_labels, func=nn.CrossEntropyLoss()):
        '''loss func for NER tasks
            out is logit from the model. Shape (bs, seq_len, hidden_dim[num_labels])
            target is target from dataloader. Shape (bs, seq_len)
        '''
        #the mask tell us where non zero tokens are
        #the num_labels is used to tell us how many labels(le.classes_) are in the targ
        non_zero_tokens = mask.view(-1) == 1 # zeroed token_ids have a mask of 1
        ignore_index = func.ignore_index

    #     if the token is not zero, select the corresponding target else set ignore_index
        cleaned_target = torch.where(non_zero_tokens.to(target.device), 
                                     target.view(-1), 
                                     torch.tensor(ignore_index).to(target.device)) #[bs*seq_len]
        
        cleaned_out = out.view(-1, num_labels) #[bs*seq_len, num_labels]
        
        loss = func(cleaned_out, cleaned_target)

        return loss
    
    @staticmethod
    def plot_loss_update(epoch, epochs, mb, train_loss, valid_loss):
        """ dynamically print the loss plot during the training/validation loop.
            expects epoch to start from 1.
        """
        x = range(1, epoch+1)
        y = np.concatenate((train_loss, valid_loss))
        graphs = [[x,train_loss], [x,valid_loss]]
        x_margin = 0.2
        y_margin = 0.05
        x_bounds = [1-x_margin, epochs+x_margin]
        y_bounds = [np.min(y)-y_margin, np.max(y)+y_margin]

        mb.update_graph(np.array(graphs), np.array(x_bounds), np.array(y_bounds))

In [ ]:
torch.nonzero??